# Draft NXmpes definition

In this script I tried to create a file that contains the minimal set of information of a MPES experiment oriented to automatic analysis, as a demonstrator of a draft definition.

Here we define the "super definition" trying to encompass and allow interoperability across the whole ARPES community. Our working idea (to be discussed) is that it is possible to "layer" definitions of increasing specificity to maximize interoperability without limiting the capabilities of software based on this structure.

A basic visualization tool would then look for the NXmpes definition, allowing to open and go through essentially any output of an arpes experiment in volumetric form. More specific analysis tools may then look for more stringent definitions such as, for example:
root.entry.definition_flavor=NXfield('NXmpes_tr') or root.entry.definition_facility=NXfield('NXmpes_FLASH')
and thus achieve automated access to a wider set of metadata.

In [2]:
import h5py
import numpy as np
import os
import six
import pytest
from nexusformat.nexus import *
def printname(name):
    print(name)

# Read data and metadata from hdf5 files

In [3]:
#Reading the volumetric binned data from hdf5 file
in_d = h5py.File('Original Files/201805_WSe2_LCPpump_LPprobe.h5', 'r')
#Show hdf5 hierarchy (dictionary-like syntax)
in_d.visit(printname)

axes
axes/E
axes/kx
axes/ky
axes/tpp
binned
binned/V


In [4]:
#reduce dimension to demonstrate the acceptable dataset with the smallest dimensionality (2D)
low_D_data=in_d['binned']['V'][:,38:42,:,:].sum(axis=(1,3))

# Create NXentry object

In [5]:
root=NXroot(NXentry())

In [6]:
root.entry.definition=NXfield('NXmpes') 

   ## Create NXinstrument object

In [7]:
root.entry.instrument=NXinstrument()

### Create beam_probe_0 object

In [8]:
root.entry.instrument.beam_probe_0=NXbeam()

In [9]:
root.entry.instrument.beam_probe_0.distance=NXfield(0)
root.entry.instrument.beam_probe_0.photon_energy=NXfield(36.49699020385742, units='eV')

### Create manipulator:NXpositioner object 

In [10]:
root.entry.instrument.manipulator=NXpositioner() # Details of the substructure of manipulator are not required

### Create NXdetector object

In [11]:
root.entry.instrument.analyser=NXdetector()

In [12]:
root.entry.instrument.analyser.lens_mode=NXfield('20180430_KPEEM_M_-2.5_FoV6.2_rezAA_20ToF_focused.sav')
root.entry.instrument.analyser.pass_energy=NXfield(20, units='eV') # Pass energy for hemispherical, drift energy for a tof
root.entry.instrument.analyser.energy_scan_mode=NXfield('fixed')

## Create NXsample object

In [13]:
root.entry.sample=NXsample()

In [14]:
root.entry.sample.name=NXfield('WSe2')
root.entry.sample.temperature=NXfield(300, units='K')
root.entry.sample.pressure=NXfield(3.85e-10, units='mbar')

## Create NXdata object

In [15]:
# Create the NXdata object
#Build the axes with names and units
kx=NXfield(in_d['axes']['kx'],name='k_x',units='1/Å')
E=NXfield(in_d['axes']['E'],name='Energy',units='eV')
data_vol=NXfield(low_D_data,name='Photoemission intensity',units='counts', compression="gzip", compression_opts=0)

In [16]:
# Wrap the NXfields in a NXdata object
root.entry.data=NXdata(data_vol,[kx,E])

# Save File

In [17]:
root.save('201805_WSe2_min.nxs', mode='w')

NXroot('root')